In [1]:
#import pandas as pd
import numpy as np
import os
import keras
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.layers import Dense,GlobalAveragePooling2D,Dropout
from tensorflow.keras.applications import MobileNet
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input, MobileNetV2
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam


Using TensorFlow backend.


In [5]:
base_model=MobileNetV2(weights='imagenet',include_top=False, input_shape=(224,224,3))

x=base_model.output
x=GlobalAveragePooling2D()(x)
x=Dense(1024,activation='relu')(x)
x=Dropout(0.3)(x)
x=Dense(1024,activation='relu')(x)
x=Dropout(0.3)(x)
x=Dense(512,activation='relu')(x)
x=Dropout(0.5)(x)
preds=Dense(51,activation='softmax')(x)

9412608/9406464 [==============================] - 11s 1us/step


In [6]:

model=Model(inputs=base_model.input,outputs=preds)


In [7]:

for i,layer in enumerate(model.layers):
  print(i,layer.name)


0 input_1
1 Conv1_pad
2 Conv1
3 bn_Conv1
4 Conv1_relu
5 expanded_conv_depthwise
6 expanded_conv_depthwise_BN
7 expanded_conv_depthwise_relu
8 expanded_conv_project
9 expanded_conv_project_BN
10 block_1_expand
11 block_1_expand_BN
12 block_1_expand_relu
13 block_1_pad
14 block_1_depthwise
15 block_1_depthwise_BN
16 block_1_depthwise_relu
17 block_1_project
18 block_1_project_BN
19 block_2_expand
20 block_2_expand_BN
21 block_2_expand_relu
22 block_2_depthwise
23 block_2_depthwise_BN
24 block_2_depthwise_relu
25 block_2_project
26 block_2_project_BN
27 block_2_add
28 block_3_expand
29 block_3_expand_BN
30 block_3_expand_relu
31 block_3_pad
32 block_3_depthwise
33 block_3_depthwise_BN
34 block_3_depthwise_relu
35 block_3_project
36 block_3_project_BN
37 block_4_expand
38 block_4_expand_BN
39 block_4_expand_relu
40 block_4_depthwise
41 block_4_depthwise_BN
42 block_4_depthwise_relu
43 block_4_project
44 block_4_project_BN
45 block_4_add
46 block_5_expand
47 block_5_expand_BN
48 block_5_exp

In [8]:
for layer in model.layers[:156]:
    layer.trainable=False
for layer in model.layers[156:]:
    layer.trainable=True


In [2]:
batch_size = 32
IMG_SIZE = 224
DATADIR = "/home/rekca/Datasets/Caltech/CaltechAnimals (copy 1)/256_ObjectCategories/"
'''
train_datagen = ImageDataGenerator( rotation_range=60, width_shift_range=0.1,
                            height_shift_range=0.1, shear_range=0.2,
                            zoom_range=0.2, horizontal_flip = True,
                           fill_mode ='nearest', rescale=1./255, validation_split=0.1)
'''

"""
train_datagen = ImageDataGenerator( validation_split=0.1,preprocessing_function=preprocess_input,
                                  horizontal_flip=True,rotation_range=40, width_shift_range=0.1,
                                   height_shift_range=0.1, shear_range=0.2,zoom_range=0.2,fill_mode ='nearest')
"""
train_datagen = ImageDataGenerator( validation_split=0.1,preprocessing_function=preprocess_input)

train_generator = train_datagen.flow_from_directory(
        DATADIR,  
        target_size=(IMG_SIZE, IMG_SIZE),  
        color_mode='rgb',
        batch_size=batch_size,
        class_mode='categorical',
        shuffle=True, 
        subset='training') 

validation_generator = train_datagen.flow_from_directory(
        DATADIR,
        target_size=(IMG_SIZE,IMG_SIZE),
        color_mode='rgb',
        batch_size=batch_size,
        class_mode='categorical',
        shuffle = True,
        subset='validation')

Found 5238 images belonging to 51 classes.
Found 562 images belonging to 51 classes.


In [14]:
model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])

In [15]:
filepath = "mobile2.{epoch:02d}-{val_acc:.2f}.hdf5" 

checkpoint = tf.keras.callbacks.ModelCheckpoint(filepath, monitor='val_loss', save_best_only=True,
                            mode='min', period=1)

callBacks = [checkpoint]

In [17]:
model.fit_generator(
        train_generator,
        steps_per_epoch=6000 // batch_size,
        epochs=10,
        validation_data=validation_generator,
        validation_steps=500 // batch_size, callbacks = callBacks)


Epoch 1/10
187/187 [==============================] - 470s 3s/step - loss: 0.7664 - acc: 0.7927 - val_loss: 0.5294 - val_acc: 0.8562
Epoch 2/10
187/187 [==============================] - 465s 2s/step - loss: 0.6472 - acc: 0.8158 - val_loss: 0.4667 - val_acc: 0.8896
Epoch 3/10
187/187 [==============================] - 467s 2s/step - loss: 0.5696 - acc: 0.8439 - val_loss: 0.4922 - val_acc: 0.8708
Epoch 4/10
187/187 [==============================] - 473s 3s/step - loss: 0.4853 - acc: 0.8649 - val_loss: 0.5416 - val_acc: 0.8688
Epoch 5/10
187/187 [==============================] - 480s 3s/step - loss: 0.4823 - acc: 0.8693 - val_loss: 0.5127 - val_acc: 0.8792
Epoch 6/10
187/187 [==============================] - 453s 2s/step - loss: 0.4144 - acc: 0.8889 - val_loss: 0.4202 - val_acc: 0.8875
Epoch 7/10
187/187 [==============================] - 442s 2s/step - loss: 0.3796 - acc: 0.8944 - val_loss: 0.5323 - val_acc: 0.8750
Epoch 8/10
187/187 [==============================] - 423s 2s/step - 

In [40]:
model.save("MobileNetV2.model")

In [13]:
converter = tf.contrib.lite.TFLiteConverter.from_keras_model_file('mobile2.04-0.33.hdf5')
tflite_model = converter.convert()
open("Mobile2.tflite", "wb").write(tflite_model)


INFO:tensorflow:Froze 1132 variables.
INFO:tensorflow:Converted 1132 variables to const ops.


27780024

In [6]:
label_map = (validation_generator.class_indices)
label_map = dict((v,k) for k,v in label_map.items()) #flip k,v

print(label_map)

{0: '007.bat', 1: '009.bear', 2: '024.butterfly', 3: '028.camel', 4: '034.centipede', 5: '038.chimp', 6: '040.cockroach', 7: '049.cormorant', 8: '052.crab-101', 9: '056.dog', 10: '057.dolphin-101', 11: '060.duck', 12: '064.elephant-101', 13: '065.elk', 14: '080.frog', 15: '084.giraffe', 16: '085.goat', 17: '087.goldfish', 18: '089.goose', 19: '090.gorilla', 20: '093.grasshopper', 21: '100.hawksbill-101', 22: '105.horse', 23: '106.horseshoe-crab', 24: '111.house-fly', 25: '113.hummingbird', 26: '114.ibis-101', 27: '116.iguana', 28: '121.kangaroo-101', 29: '124.killer-whale', 30: '129.leopards-101', 31: '134.llama-101', 32: '150.octopus', 33: '151.ostrich', 34: '152.owl', 35: '158.penguin', 36: '159.people', 37: '164.porcupine', 38: '166.praying-mantis', 39: '168.raccoon', 40: '179.scorpion-101', 41: '186.skunk', 42: '189.snail', 43: '190.snake', 44: '198.spider', 45: '201.starfish-101', 46: '207.swan', 47: '228.triceratops', 48: '230.trilobite-101', 49: '250.zebra', 50: '256.toad'}
